<a id="top"></a>
# Benchmark Tool

## About Benchmark Tool

This sample provides a comparison of performance benchmarks using the [OpenVINO Benchmarking app](https://docs.openvinotoolkit.org/latest/_inference_engine_tools_benchmark_tool_README.html). The Benchmark python tool provides estimation of deep learning inference performance on the supported devices. 

Upon start-up, the application reads command-line parameters and loads a network and images/binary files to the Inference Engine plugin, which is chosen depending on a specified device. 

###  Setup the environment variables and import dependencies

In [ ]:
from IPython.display import HTML
import os
import time
import sys                                                     
from openvino.inference_engine import IECore
import cv2
import pandas as pd
from qarpo.model_visualizer_link import *
from qarpo.demoutils import liveQstat
from job_utils import create_jobs_from_config, wait_for_job_to_finish

### Install progress package


In [ ]:
!pip3 install -r ./benchmark/requirements.txt

### Deep Learning model for inference
This example uses a Tensorflow* implementation of a resnet-50 model for classification.

#### Download the resnet-50 Tensorflow* model from the model downloader from the Intel distribution OpenVINO toolkit


In [ ]:
!omz_downloader --name resnet-50-tf -o models

To view a graph of the model used in this application, run the cell below then select the link generated:

In [ ]:
showModelVisualizerLink("models/public/resnet-50-tf/resnet_v1-50.pb")

### Optimize a deep-learning model using the Model Optimizer (MO) 
In this section, you will use the Model Optimizer to convert a trained model to two Intermediate Representation (IR) files (one .bin and one .xml). The Inference Engine requires this model conversion so that it can use the IR as input and achieve optimum performance on Intel® hardware.

#### Create a directory to store IR files

In [ ]:
! mkdir -p models/FP32
! mkdir -p models/FP16

##### Convert the model with FP16 quantization 

In [ ]:
!mo \
--input_model models/public/resnet-50-tf/resnet_v1-50.pb \
--input_shape=[1,224,224,3] \
--mean_values=[123.68,116.78,103.94] \
-o models/FP16 \
--data_type FP16

##### Convert the model with FP32 quantization 

In [ ]:
!mo \
--input_model models/public/resnet-50-tf/resnet_v1-50.pb \
--input_shape=[1,224,224,3] \
--mean_values=[123.68,116.78,103.94] \
-o models/FP32 \
--data_type FP32

### Creating job file
Till now, we ran all the above steps on a single edge system allocated for your account. Now we want to run the inference on different edge systems on the Intel IoT devcloud to benchmark the inference performance. For that, we will submit the inference jobs for each edge device in a queue. For each job, we will specify the type of the edge compute node that must be allocated for the job.

The job file in the below cell is written in Bash, and will be executed directly on the edge compute node. Run the following cell to write this in to the file "benchmark_app_job.sh"

In [ ]:
%%writefile benchmark_app_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
JOB_ID=`basename ${0} | cut -f1 -d"."`
OUTPUT_FILE=$1
DEVICE=$2
API=$3

echo VENV_PATH=$VENV_PATH
echo OPENVINO_RUNTIME=$OPENVINO_RUNTIME

# Follow this order of setting up environment for openVINO 2022.1.0.553
echo "Activating a Python virtual environment from ${VENV_PATH}..."
source ${VENV_PATH}/bin/activate
echo "Activating OpenVINO variables from ${OPENVINO_RUNTIME}..."
source ${OPENVINO_RUNTIME}/setupvars.sh

# Benchmark Application script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $OUTPUT_FILE

SAMPLEPATH=$PBS_O_WORKDIR

mkdir -p ${OUTPUT_FILE}
rm -f ${OUTPUT_FILE}/*

FP_MODEL="FP16"

# Running the benchmark application code

python3 benchmark_app.py -m ${SAMPLEPATH}/models/${FP_MODEL}/resnet_v1-50.xml \
            -d $DEVICE \
            -niter 10 \
            -api $API \
            --report_type detailed_counters \
            --report_folder ${SAMPLEPATH}/${OUTPUT_FILE}


## Benchmark systems with the deep learning model
The following code block creates job execution commands for the systems listed in the job_config.json file. The jobs will be queued and then run when an instance of the hardware is available.

In [ ]:
# import necessary libraries
import json
import qarpo

# load job configurations for demo
with open('job_config.json') as json_file:  
    data = json.load(json_file)
    
# create and run the user job interface
job_interface = qarpo.Interface(data)
job_interface.displayUI()

### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command.  The custom Jupyter* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live updates.  

Run the following cell to display the current job status with periodic updates. 
You should see the jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit the jobs in the previous step).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running.

The `S` column shows the current status of each job: 
- If the status is `Q`, then the job is queued and waiting for available resources
- If the status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

In [ ]:
liveQstat()

# Next steps
- [More Jupyter* Notebook Samples](https://software.intel.com/content/www/us/en/develop/tools/devcloud/edge/build/sample-apps.html) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://software.intel.com/content/www/us/en/develop/tools/devcloud/edge/learn/tutorials.html) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


# About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://community.intel.com/t5/Intel-DevCloud/bd-p/devcloud)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>